In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
import json
import time

# Load saved cookies from the JSON file
with open('cookies.json', 'r') as file:
    cookies = json.load(file)

# Start a new Firefox WebDriver session
driver = webdriver.Firefox()

# Open the base domain (needed to add cookies)
driver.get('https://pony.town')

# Add the cookies to the current session
for cookie in cookies:
    if 'sameSite' in cookie and cookie['sameSite'] not in ["Strict", "Lax", "None"]:
        cookie['sameSite'] = 'None'  # Adjust 'sameSite' attribute if necessary
    try:
        driver.add_cookie(cookie)
    except Exception as e:
        print(f"Error adding cookie: {cookie['name']}, {e}")

# Navigate to the specific page (search query)
driver.get('https://pony.town/')

# Allow time for the page to load
WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'btn-success')))

# Locate the button by its class name and text content
button = WebDriverWait(driver, 20).until(
    EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'btn-success') and contains(., 'Play')]"))
)

# Click the button
button.click()

time.sleep(5)

# Press the 'O' key
webdriver.ActionChains(driver).send_keys('O').perform()

# JavaScript code to use MutationObserver to watch for new chat lines and extract details
mutation_observer_script = """
const chatLines = [];
const observer = new MutationObserver((mutations) => {
    mutations.forEach((mutation) => {
        mutation.addedNodes.forEach((node) => {
            if (node.classList && node.classList.contains('chat-line')) {
                const now = new Date();
                const time = new Intl.DateTimeFormat('en-US', {
                    timeZone: 'America/New_York',
                    year: 'numeric',
                    month: '2-digit',
                    day: '2-digit',
                    hour: '2-digit',
                    minute: '2-digit',
                    second: '2-digit',
                    hour12: false
                }).format(now);
                const name = node.querySelector('.chat-line-name-content').innerText;
                const message = node.querySelector('.chat-line-message').innerText;
                chatLines.push({ time, name, message });
            }
        });
    });
});
observer.observe(document.body, { childList: true, subtree: true });
window.getChatLines = () => chatLines;
"""


# Execute the JavaScript code
driver.execute_script(mutation_observer_script)

# Monitor for new chat lines for 30 seconds
time.sleep(30)

# Retrieve the chat lines collected by the MutationObserver
chat_lines = driver.execute_script("return window.getChatLines();")

# Save the chat lines to a JSON file
with open('chat_lines.json', 'w', encoding='utf-8') as file:
    json.dump(chat_lines, file, ensure_ascii=False, indent=4)

# Save the HTML content to a file
html_content = driver.page_source
with open('page_content.html', 'w', encoding='utf-8') as file:
    file.write(html_content)


# Take a screenshot as proof of session reuse
driver.save_screenshot('session_with_cookies.png')

# Quit the browser
driver.quit()

